In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# COVID-19 Global Data Tracker\n",
    "\n",
    "## 📊 Data-Driven Dashboard for Global COVID-19 Trends\n",
    "\n",
    "This notebook provides comprehensive analysis of global COVID-19 data including:\n",
    "- Case and death trends by country\n",
    "- Vaccination progress analysis\n",
    "- Country comparisons and insights\n",
    "- Interactive visualizations\n",
    "\n",
    "**Data Source:** Our World in Data COVID-19 Dataset"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📥 1. Data Collection & Setup"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import required libraries\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import plotly.express as px\n",
    "import plotly.graph_objects as go\n",
    "from plotly.subplots import make_subplots\n",
    "import warnings\n",
    "from datetime import datetime, timedelta\n",
    "import requests\n",
    "from io import StringIO\n",
    "\n",
    "# Configuration\n",
    "warnings.filterwarnings('ignore')\n",
    "plt.style.use('seaborn-v0_8')\n",
    "sns.set_palette(\"husl\")\n",
    "pd.set_option('display.max_columns', None)\n",
    "pd.set_option('display.max_rows', 100)\n",
    "\n",
    "print(\"✅ Libraries imported successfully!\")\n",
    "print(f\"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load COVID-19 data from Our World in Data\n",
    "print(\"📥 Loading COVID-19 data...\")\n",
    "\n",
    "# Try to load from URL first, fallback to local file\n",
    "try:\n",
    "    url = \"https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv\"\n",
    "    df_raw = pd.read_csv(url)\n",
    "    print(f\"✅ Data loaded from URL: {df_raw.shape[0]:,} rows, {df_raw.shape[1]} columns\")\n",
    "except:\n",
    "    print(\"⚠️ Unable to load from URL, using local file...\")\n",
    "    df_raw = pd.read_csv('owid-covid-data.csv')\n",
    "    print(f\"✅ Data loaded from local file: {df_raw.shape[0]:,} rows, {df_raw.shape[1]} columns\")\n",
    "\n",
    "# Display basic information\n",
    "print(f\"\\n📊 Dataset Info:\")\n",
    "print(f\"Date range: {df_raw['date'].min()} to {df_raw['date'].max()}\")\n",
    "print(f\"Countries: {df_raw['location'].nunique()}\")\n",
    "print(f\"Total records: {len(df_raw):,}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📊 2. Data Loading & Exploration"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Explore data structure\n",
    "print(\"🔍 Data Structure Overview:\")\n",
    "print(\"\\n\" + \"=\"*50)\n",
    "print(\"DATASET SHAPE:\")\n",
    "print(f\"Rows: {df_raw.shape[0]:,}\")\n",
    "print(f\"Columns: {df_raw.shape[1]}\")\n",
    "\n",
    "print(\"\\n\" + \"=\"*50)\n",
    "print(\"FIRST 5 ROWS:\")\n",
    "display(df_raw.head())\n",
    "\n",
    "print(\"\\n\" + \"=\"*50)\n",
    "print(\"COLUMN INFO:\")\n",
    "df_raw.info()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check missing values for key columns\n",
    "key_columns = [\n",
    "    'date', 'location', 'total_cases', 'total_deaths', 'new_cases', \n",
    "    'new_deaths', 'total_vaccinations', 'people_vaccinated', 'population'\n",
    "]\n",
    "\n",
    "print(\"🔍 Missing Values Analysis:\")\n",
    "print(\"\\n\" + \"=\"*50)\n",
    "\n",
    "missing_data = df_raw[key_columns].isnull().sum().sort_values(ascending=False)\n",
    "missing_pct = (missing_data / len(df_raw) * 100).round(2)\n",
    "\n",
    "missing_df = pd.DataFrame({\n",
    "    'Missing Count': missing_data,\n",
    "    'Missing %': missing_pct\n",
    "})\n",
    "\n",
    "display(missing_df[missing_df['Missing Count'] > 0])\n",
    "\n",
    "# Display unique locations sample\n",
    "print(\"\\n\" + \"=\"*50)\n",
    "print(\"SAMPLE LOCATIONS:\")\n",
    "locations = df_raw['location'].unique()\n",
    "print(f\"Total locations: {len(locations)}\")\n",
    "print(\"Sample:\", locations[:20])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🧹 3. Data Cleaning"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Select focus countries for analysis\n",
    "focus_countries = ['United States', 'India', 'Kenya', 'Germany', 'Brazil']\n",
    "\n",
    "print(f\"🎯 Focus Countries: {', '.join(focus_countries)}\")\n",
    "print(\"\\n\" + \"=\"*50)\n",
    "\n",
    "# Filter data for focus countries\n",
    "df_focus = df_raw[df_raw['location'].isin(focus_countries)].copy()\n",
    "print(f\"✅ Filtered data: {len(df_focus):,} rows for {df_focus['location'].nunique()} countries\")\n",
    "\n",
    "# Convert date to datetime\n",
    "df_focus['date'] = pd.to_datetime(df_focus['date'])\n",
    "print(f\"✅ Date converted to datetime format\")\n",
    "\n",
    "# Sort by location and date\n",
    "df_focus = df_focus.sort_values(['location', 'date']).reset_index(drop=True)\n",
    "print(f\"✅ Data sorted by location and date\")\n",
    "\n",
    "# Check data availability for each country\n",
    "print(\"\\n📊 Data Availability by Country:\")\n",
    "for country in focus_countries:\n",
    "    country_data = df_focus[df_focus['location'] == country]\n",
    "    date_range = f\"{country_data['date'].min().strftime('%Y-%m-%d')} to {country_data['date'].max().strftime('%Y-%m-%d')}\"\n",
    "    print(f\"{country:15}: {len(country_data):4} records | {date_range}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Clean and prepare key metrics\n",
    "print(\"🧹 Cleaning Key Metrics...\")\n",
    "\n",
    "# Fill missing values using forward fill method\n",
    "numeric_columns = ['total_cases', 'total_deaths', 'total_vaccinations', 'people_vaccinated']\n",
    "\n",
    "for col in numeric_columns:\n",
    "    if col in df_focus.columns:\n",
    "        # Forward fill within each country\n",
    "        df_focus[col] = df_focus.groupby('location')[col].fillna(method='ffill')\n",
    "        # Fill remaining NaN with 0\n",
    "        df_focus[col] = df_focus[col].fillna(0)\n",
    "\n",
    "# Calculate derived metrics\n",
    "print(\"📈 Calculating Derived Metrics...\")\n",
    "\n",
    "# Death rate\n",
    "df_focus['death_rate'] = np.where(\n",
    "    df_focus['total_cases'] > 0,\n",
    "    (df_focus['total_deaths'] / df_focus['total_cases'] * 100).round(2),\n",
    "    0\n",
    ")\n",
    "\n",
    "# Vaccination percentage\n",
    "df_focus['vaccination_rate'] = np.where(\n",
    "    df_focus['population'] > 0,\n",
    "    (df_focus['people_vaccinated'] / df_focus['population'] * 100).round(2),\n",
    "    0\n",
    ")\n",
    "\n",
    "# Cases per million\n",
    "df_focus['cases_per_million'] = np.where(\n",
    "    df_focus['population'] > 0,\n",
    "    (df_focus['total_cases'] / df_focus['population'] * 1_000_000).round(0),\n",
    "    0\n",
    ")\n",
    "\n",
    "print(\"✅ Data cleaning completed!\")\n",
    "print(f\"\\n📊 Final Dataset: {len(df_focus):,} rows, {len(df_focus.columns)} columns\")\n",
    "print(f\"📅 Date Range: {df_focus['date'].min().strftime('%Y-%m-%d')} to {df_focus['date'].max().strftime('%Y-%m-%d')}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔍 4. Exploratory Data Analysis (EDA)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get latest data for each country\n",
    "latest_data = df_focus.groupby('location').last().reset_index()\n",
    "\n",
    "print(\"📊 Latest COVID-19 Statistics by Country:\")\n",
    "print(\"=\"*80)\n",
    "\n",
    "summary_cols = ['location', 'total_cases', 'total_deaths', 'death_rate', \n",
    "                'people_vaccinated', 'vaccination_rate', 'population']\n",
    "\n",
    "summary_df = latest_data[summary_cols].copy()\n",
    "summary_df['total_cases'] = summary_df['total_cases'].apply(lambda x: f\"{x:,.0f}\")\n",
    "summary_df['total_deaths'] = summary_df['total_deaths'].apply(lambda x: f\"{x:,.0f}\")\n",
    "summary_df['people_vaccinated'] = summary_df['people_vaccinated'].apply(lambda x: f\"{x:,.0f}\")\n",
    "summary_df['population'] = summary_df['population'].apply(lambda x: f\"{x:,.0f}\")\n",
    "\n",
    "display(summary_df)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot 1: Total Cases Over Time\n",
    "plt.figure(figsize=(15, 8))\n",
    "\n",
    "for country in focus_countries:\n",
    "    country_data = df_focus[df_focus['location'] == country]\n",
    "    plt.plot(country_data['date'], country_data['total_cases'], \n",
    "             marker='o', markersize=3, linewidth=2, label=country)\n",
    "\n",
    "plt.title('Total COVID-19 Cases Over Time by Country', fontsize=16, fontweight='bold', pad=20)\n",
    "plt.xlabel('Date', fontsize=12)\n",
    "plt.ylabel('Total Cases', fontsize=12)\n",
    "plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.yscale('log')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"📈 This log-scale chart shows the exponential growth patterns of COVID-19 cases across countries.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot 2: Deaths Over Time\n",
    "plt.figure(figsize=(15, 8))\n",
    "\n",
    "for country in focus_countries:\n",
    "    country_data = df_focus[df_focus['location'] == country]\n",
    "    plt.plot(country_data['date'], country_data['total_deaths'], \n",
    "             marker='s', markersize=3, linewidth=2, label=country)\n",
    "\n",
    "plt.title('Total COVID-19 Deaths Over Time by Country', fontsize=16, fontweight='bold', pad=20)\n",
    "plt.xlabel('Date', fontsize=12)\n",
    "plt.ylabel('Total Deaths', fontsize=12)\n",
    "plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.yscale('log')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"💀 Death trends often lag case trends by 2-4 weeks due to disease progression.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot 3: Daily New Cases (Moving Average)\n",
    "plt.figure(figsize=(15, 8))\n",
    "\n",
    "for country in focus_countries:\n",
    "    country_data = df_focus[df_focus['location'] == country].copy()\n",
    "    # Calculate 7-day moving average for smoother trends\n",
    "    country_data['new_cases_7day'] = country_data['new_cases'].rolling(window=7, center=True).mean()\n",
    "    \n",
    "    plt.plot(country_data['date'], country_data['new_cases_7day'], \n",
    "             linewidth=2, label=f\"{country} (7-day avg)\")\n",
    "\n",
    "plt.title('Daily New COVID-19 Cases (7-Day Moving Average)', fontsize=16, fontweight='bold', pad=20)\n",
    "plt.xlabel('Date', fontsize=12)\n",
    "plt.ylabel('Daily New Cases (7-day average)', fontsize=12)\n",
    "plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"📊 Moving averages help identify true trends by smoothing daily reporting variations.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot 4: Bar Chart - Latest Total Cases\n",
    "plt.figure(figsize=(12, 8))\n",
    "\n",
    "bars = plt.bar(latest_data['location'], latest_data['total_cases'], \n",
    "               color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7'])\n",
    "\n",
    "plt.title('Total COVID-19 Cases by Country (Latest Data)', fontsize=16, fontweight='bold', pad=20)\n",
    "plt.xlabel('Country', fontsize=12)\n",
    "plt.ylabel('Total Cases', fontsize=12)\n",
    "plt.xticks(rotation=45)\n",
    "plt.grid(True, alpha=0.3, axis='y')\n",
    "\n",
    "# Add value labels on bars\n",
    "for bar, value in zip(bars, latest_data['total_cases']):\n",
    "    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + bar.get_height()*0.01,\n",
    "             f'{value:,.0f}', ha='center', va='bottom', fontweight='bold')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"📊 The United States and India have had the highest total case counts globally.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 💉 5. Vaccination Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot 5: Vaccination Progress Over Time\n",
    "plt.figure(figsize=(15, 8))\n",
    "\n",
    "for country in focus_countries:\n",
    "    country_data = df_focus[df_focus['location'] == country]\n",
    "    # Filter data where vaccination data exists\n",
    "    vacc_data = country_data[country_data['vaccination_rate'] > 0]\n",
    "    \n",
    "    if len(vacc_data) > 0:\n",
    "        plt.plot(vacc_data['date'], vacc_data['vaccination_rate'], \n",
    "                 marker='o', markersize=3, linewidth=2, label=country)\n",
    "\n",
    "plt.title('COVID-19 Vaccination Progress (% Population Vaccinated)', fontsize=16, fontweight='bold', pad=20)\n",
    "plt.xlabel('Date', fontsize=12)\n",
    "plt.ylabel('Vaccination Rate (% of Population)', fontsize=12)\n",
    "plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.ylim(0, 100)\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"💉 Vaccination campaigns started in early 2021 with varying rollout speeds across countries.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot 6: Current Vaccination Status (Pie Charts)\n",
    "fig, axes = plt.subplots(2, 3, figsize=(18, 12))\n",
    "axes = axes.flatten()\n",
    "\n",
    "for i, country in enumerate(focus_countries):\n",
    "    country_latest = latest_data[latest_data['location'] == country]\n",
    "    \n",
    "    if len(country_latest) > 0:\n",
    "        vacc_rate = country_latest['vaccination_rate'].iloc[0]\n",
    "        unvacc_rate = 100 - vacc_rate\n",
    "        \n",
    "        if vacc_rate > 0:\n",
    "            sizes = [vacc_rate, unvacc_rate]\n",
    "            labels = [f'Vaccinated\\n{vacc_rate:.1f}%', f'Unvaccinated\\n{unvacc_rate:.1f}%']\n",
    "            colors = ['#4ECDC4', '#FFE5E5']\n",
    "            \n",
    "            axes[i].pie(sizes, labels=labels, colors=colors, autopct='', startangle=90)\n",
    "            axes[i].set_title(f'{country}\\nVaccination Status', fontweight='bold')\n",
    "        else:\n",
    "            axes[i].text(0.5, 0.5, f'{country}\\nNo vaccination\\ndata available', \n",
    "                        ha='center', va='center', transform=axes[i].transAxes)\n",
    "            axes[i].set_xlim(0, 1)\n",
    "            axes[i].set_ylim(0, 1)\n",
    "\n",
    "# Remove the empty subplot\n",
    "fig.delaxes(axes[5])\n",
    "\n",
    "plt.suptitle('Current Vaccination Status by Country', fontsize=16, fontweight='bold')\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"💉 Vaccination rates vary significantly across countries based on supply, infrastructure, and policy.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🌍 6. Interactive Visualizations with Plotly"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Interactive Plot 1: Cases and Deaths Over Time\n",
    "fig = make_subplots(\n",
    "    rows=2, cols=1,\n",
    "    subplot_titles=('Total Cases Over Time', 'Total Deaths Over Time'),\n",
    "    vertical_spacing=0.1\n",
    ")\n",
    "\n",
    "colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']\n",
    "\n",
    "for i, country in enumerate(focus_countries):\n",
    "    country_data = df_focus[df_focus['location'] == country]\n",
    "    \n",
    "    # Cases\n",
    "    fig.add_trace(\n",
    "        go.Scatter(\n",
    "            x=country_data['date'],\n",
    "            y=country_data['total_cases'],\n",
    "            mode='lines',\n",
    "            name=f'{country} Cases',\n",
    "            line=dict(color=colors[i], width=3),\n",
    "            hovertemplate=f'<b>{country}</b><br>Date: %{x}<br>Cases: %{y:,.0f}<extra></extra>'\n",
    "        ),\n",
    "        row=1, col=1\n",
    "    )\n",
    "    \n",
    "    # Deaths\n",
    "    fig.add_trace(\n",
    "        go.Scatter(\n",
    "            x=country_data['date'],\n",
    "            y=country_data['total_deaths'],\n",
    "            mode='lines',\n",
    "            name=f'{country} Deaths',\n",
    "            line=dict(color=colors[i], width=3, dash='dash'),\n",
    "            hovertemplate=f'<b>{country}</b><br>Date: %{x}<br>Deaths: %{y:,.0f}<extra></extra>'\n",
    "        ),\n",
    "        row=2, col=1\n",
    "    )\n",
    "\n",
    "fig.update_layout(\n",
    "    height=800,\n",
    "    title_text=\"Interactive COVID-19 Trends Analysis\",\n",
    "    title_x=0.5,\n",
    "    showlegend=True\n",
    ")\n",
    "\n",
    "fig.update_yaxes(type=\"log\", row=1, col=1)\n",
    "fig.update_yaxes(type=\"log\", row=2, col=1)\n",
    "\n",
    "fig.show()\n",
    "\n",
    "print(\"🖱️ Interactive chart: Hover for details, zoom, and toggle country visibility!\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Interactive Plot 2: Vaccination vs Cases Correlation\n",
    "fig = px.scatter(\n",
    "    latest_data,\n",
    "    x='vaccination_rate',\n",
    "    y='cases_per_million',\n",
    "    size='population',\n",
    "    color='location',\n",
    "    hover_name='location',\n",
    "    hover_data={\n",
    "        'total_cases': ':,.0f',\n",
    "        'total_deaths': ':,.0f',\n",
    "        'death_rate': ':.2f',\n",
    "        'population': ':,.0f'\n",
    "    },\n",
    "    title='Vaccination Rate vs Cases per Million Population',\n",
    "    labels={\n",
    "        'vaccination_rate': 'Vaccination Rate (% Population)',\n",
    "        'cases_per_million': 'Cases per Million Population'\n",
    "    }\n",
    ")\n",
    "\n",
    "fig.update_layout(\n",
    "    height=600,\n",
    "    title_x=0.5\n",
    ")\n",
    "\n",
    "fig.show()\n",
    "\n",
    "print(\"📊 Bubble size represents population. This helps identify the relationship between vaccination and case rates.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🧠 7. Key Insights & Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate comprehensive insights\n",
    "print(\"🧠 COVID-19 DATA INSIGHTS REPORT\")\n",
    "print(\"=\"*60)\n",
    "\n",
    "# Insight 1: Peak cases analysis\n",
    "print(\"\\n📈 1. PEAK CASES ANALYSIS:\")\n",
    "for country in focus_countries:\n",
    "    country_data = df_focus[df_focus['location'] == country]\n",
    "    if 'new_cases' in country_data.columns and not country_data['new_cases'].isna().all():\n",
    "        peak_idx = country_data['new_cases'].idxmax()\n",
    "        peak_date = country_data.loc[peak_idx, 'date']\n",
    "        peak_cases = country_data.loc[peak_idx, 'new_cases']\n",
    "        print(f\"   {country:15}: Peak of {peak_cases:,.0f} daily cases on {peak_date.strftime('%Y-%m-%d')}\")\n",
    "\n",
    "# Insight 2: Death rates comparison\n",
    "print(\"\\n💀 2. DEATH RATES COMPARISON:\")\n",
    "death_rates = latest_data[['location', 'death_rate']].sort_values('death_rate', ascending=False)\n",
    "for _, row in death_rates.iterrows():\n",
    "    print(f\"   {row['location']:15}: {row['death_rate']:.2f}% death rate\")\n",
    "\n",
    "# Insight 3: Vaccination leaders\n",
    "print(\"\\n💉 3. VACCINATION PROGRESS:\")\n",
    "vacc_rates = latest_data[['location', 'vaccination_rate']].sort_values('vaccination_rate', ascending=False)\n",
    "for _, row in vacc_rates.iterrows():\n",
    "    if row['vaccination_rate'] > 0:\n",
    "        print(f\"   {row['location']:15}: {row['vaccination_rate']:.1f}% population vaccinated\")\n",
    "\n",
    "# Insight 4: Population impact\n",
    "print(\"\\n🌍 4. POPULATION IMPACT:\")\n",
    "pop_impact = latest_data[['location', 'cases_per_million']].sort_values('cases_per_million', ascending=False)\n",
    "for _, row in pop_impact.iterrows():\n",
    "    print(f\"   {row['location']:15}: {row['cases_per_million']:.0f} cases per million population\")\n",
    "\n",
    "# Insight 5: Timeline analysis\n",
    "print(\"\\n📅 5. TIMELINE INSIGHTS:\")\n",
    "first_case_dates = df_focus[df_focus['total_cases'] > 0].groupby('location')['date'].min()\n",
    "for country in focus_countries:\n",
    "    if country in first_case_dates.index:\n",
    "        first_case = first_case_dates[country]\n",
    "        days_since = (df_focus['date'].max() - first_case).days\n",
    "        print(f\"   {country:15}: First case {first_case.strftime('%Y-%m-%d')} ({days_since} days ago)\")\n",
    "\n",
    "print(\"\\n\" + \"=\"*60)\n",
    "print(\"✅ INSIGHTS ANALYSIS COMPLETED\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📋 8. Summary Statistics & Correlation Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Correlation matrix for key metrics\n",
    "plt.figure(figsize=(10, 8))\n",
    "\n",
    "correlation_cols = ['total_cases', 'total_deaths', 'death_rate', 'vaccination_rate', 'cases_per_million']\n",
    "corr_data = latest_data[correlation_cols].corr()\n",
    "\n",
    "mask = np.triu(np.ones_like(corr_data, dtype=bool))\n",
    "sns.heatmap(\n",
    "    corr_data, \n",
    "    mask=mask,\n",
    "    annot=True, \n",
    "    cmap='RdYlBu_r', \n",
    "    center=0,\n",
    "    square=True,\n",
    "    fmt='.2f',\n",
    "    cbar_kws={'shrink': 0.8}\n",
    ")\n",
    "\n",
    "plt.title('Correlation Matrix: COVID-19 Metrics', fontsize=14, fontweight='bold', pad=20)\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"🔗 Correlation analysis helps identify relationships between different COVID-19 metrics.\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Final summary statistics\n",
    "print(\"📊 FINAL SUMMARY STATISTICS\")\n",
    "print(\"=\"*50)\n",
    "\n",
    "# Global totals for focus countries\n",
    "total_cases = latest_data['total_cases'].sum()\n",
    "total_deaths = latest_data['total_deaths'].sum()\n",
    "total_population = latest_data['population'].sum()\n",
    "avg_vaccination_rate = latest_data['vaccination_rate'].mean()\n",
    "\n",
    "print(f\"\\n🌍 AGGREGATE STATISTICS (Focus Countries):\")\n",
    "print(f\"   Total Cases:           {total_cases:>15,.0f}\")\n",
    "print(f\"   Total Deaths:          {total_deaths:>15,.0f}\")\n",
    "print(f\"   Combined Population:   {total_population:>15,.0f}\")\n",
    "print(f\"   Overall Death Rate:    {(total_deaths/total_cases*100):>15.2f}%\")\n",
    "print(f\"   Avg Vaccination Rate:  {avg_vaccination_rate:>15.1f}%\")\n",
    "\n",
    "print(f\"\\n📈 DATA PROCESSING SUMMARY:\")\n",
    "print(f\"   Original Dataset:      {len(df_raw):>15,.0f} records\")\n",
    "print(f\"   Filtered Dataset:      {len(df_focus):>15,.0f} records\")\n",
    "print(f\"   Countries Analyzed:    {len(focus_countries):>15} countries\")\n",
    "print(f\"   Date Range:            {(df_focus['date'].max() - df_focus['date'].min()).days:>15} days\")\n",
    "\n",
    "print(\"\\n\" + \"=\"*50)\n",
    "print(\"🎯 ANALYSIS COMPLETED SUCCESSFULLY!\")\n",
    "print(\"📧 Ready for transfer to VS Code and further development.\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}
